# chroma basic

[全体REF](https://dev.classmethod.jp/articles/multimodal-rag-chatbot/)

## chroma setup

In [1]:

import chromadb
chroma_client = chromadb.PersistentClient(path="./chroma_db") # PersistentClientにすると、DBがfolder保存される


## setting collection
Collectionとは、RDBでいうtableのこと。
Chromaフォルダ内でUUIDで識別される

In [2]:

import chromadb
from chromadb.utils.data_loaders import ImageLoader
from chromadb.utils.embedding_functions import OpenCLIPEmbeddingFunction
from chromadb.utils.embedding_functions import OpenCLIPEmbeddingFunction
embedding_function = OpenCLIPEmbeddingFunction() # データをAddするとき・検索するときのembedding関数の設定
data_loader = ImageLoader() # OpenCLIPでは直接画像をChroma DBに保存しない。そのため、URI指定で生画像を持ってきてくれるImage Loaderを使用する。（ただし、今回は失敗して使っていない。To be updated）

# get_collectionで既存collectionを取得。
collection = chroma_client.get_collection(
    name='multimodal_collection', # collection名
    data_loader=data_loader,
    embedding_function=embedding_function
)

# if you want to make multi modal vector database, you have two options:
# 1. use image -> captions model, then use caption to text embedding model to store chrome db.
# 2. use CLIP(Contrastive Language–Image Pretraining) embedding model to directly embed image and text into same vector space.
# CLIP leasrns a lot of image-text pairs. it is used for image search and classification, generation etc.(like DALL-E, Stable Diffusion etc.)

# 1. を使う場合、ベクトルDBに追加するデータは細切れに、元データは大枠に保存する。ベクター検索は細切れのほうが精度がよく、コンテキストとして使うには大枠のほうがいいため。
# 今回は2. を採用している。


c:\Users\ishid\miniconda3\envs\airizap\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


もしcollectionが存在しない場合、新規作成するコード例。　暴発しないようにコメントアウトしてあります。

In [ ]:
# collection = chroma_client.create_collection(
#     name='multimodal_collection',
#     embedding_function=embedding_function,
#     data_loader=data_loader
# )

for multi modal db, i refered chroma official document: 
[Embeddings - Multimodal](https://docs.trychroma.com/docs/embeddings/multimodal)

## pdf processer

In [ ]:
# PDFをテキストチャンクに分割する(pdfplumber使用)

import pdfplumber
from langchain_text_splitters import RecursiveCharacterTextSplitter

def pdf_to_text_chunks_pdfplumber(path: str):
    texts = []
    with pdfplumber.open(path) as pdf:
        for page in pdf.pages:
            texts.append(page.extract_text() or "")
    text = "\n\n".join(texts).strip()

    splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        encoding_name="cl100k_base",
        chunk_size=500,
        chunk_overlap=50,
    )
    return splitter.split_text(text)

text_chunks = pdf_to_text_chunks_pdfplumber("./.pdf/bay_ir_2025.pdf")

# REF)to deal with the pdf, you have to;
# - 1. loard pdf,
# - 2. split the pdf into chunks, 
# - 3. embed the chunks, 
# - 4. store the embedded chunks into chroma db.


In [ ]:
# TEST
text_chunks

['1/109\nEDINET提出書類\n株式会社ベイカレント(E32549)\n有価証券報告書\n【表紙】\n【提出書類】 有価証券報告書\n【根拠条文】 金融商品取引法第24条第１項\n【提出先】 関東財務局長\n【提出日】 2025年５月28日\n【事業年度】 第11期(自 2024年３月１日 至 2025年２月28日)\n【会社名】 株式会社ベイカレント\n【英訳名】 BayCurrent, Inc.\n【代表者の役職氏名】 代表取締役社長 北風 大輔\n【本店の所在の場所】 東京都港区麻布台一丁目３番１号\n【電話番号】 (０３)５５４４−９３３１(代表)\n【事務連絡者氏名】 取締役最高財務責任者 中村 公亮\n【最寄りの連絡場所】 東京都港区麻布台一丁目３番１号\n【電話番号】 (０３)５５４４−９３３１(代表)\n【事務連絡者氏名】 取締役最高財務責任者 中村 公亮\n【縦覧に供する場所】 株式会社東京証券取引所\n(東京都中央区日本橋兜町２番１号)',
 '2/109\nEDINET提出書類\n株式会社ベイカレント(E32549)\n有価証券報告書\n第一部 【企業情報】\n第１ 【企業の概況】\n１ 【主要な経営指標等の推移】\n(1) 連結経営指標等\n第７期 第８期 第９期 第10期 第11期\n回次\n−単体− −単体− −単体− −単体− −連結−\n決算年月 2021年２月 2022年２月 2023年２月 2024年２月 2025年２月\n売上収益 (百万円) 42,873 57,642 76,090 93,909 116,056\n税引前利益 (百万円) 13,477 21,469 29,875 34,160 42,546\n親会社の所有者に帰属\n(百万円) 10,014 15,544 21,910 25,382 30,760\nする当期利益\n親会社の所有者に帰属\n(百万円) 10,014 15,544 21,910 25,382 30,760\nする当期包括利益\n親会社の所有者に帰属\n(百万円) 30,018 41,701 57,341 74,127 94,401\nする持分\n資産合計 (百万円) 46,003 58,136 75,265 99,883 124,665\n１株当たり親会社\n(円) 195.81 271.

## adding document to collection

In [ ]:
# チャンクしたPDFテキストをChroma DBに保存
source = "bay_ir_2025.pdf"  # 出典名（任意）

# 空チャンクを除外（重要）
docs = [c for c in text_chunks if c and c.strip()]

ids = [f"{source}::chunk::{i}" for i in range(len(docs))]
metadatas = [{"source": source, "chunk_index": i} for i in range(len(docs))]

collection.add(
    ids=ids,
    documents=docs,
    metadatas=metadatas,
)

In [ ]:
# 画像データをChroma DBに保存
source = "bay_ir_2025.png"

from pathlib import Path
img_dir = Path("./.images")
image_paths = [(img_dir / f"00-{i:02d}.png").as_posix() for i in range(1, 38)]

ids = [f"{source}::image::{i:06d}" for i in range(len(image_paths))]
metadatas = [{"source": source, "modality": "image", "image_index": i, "path": p}
             for i, p in enumerate(image_paths)]

# 画像を「documents」に入れる場合（例：パスや説明文を入れる）
collection.add(ids=ids, documents=image_paths, metadatas=metadatas)


## コレクションのデータ一覧を取得

In [10]:
import csv
# CSVデータをChroma DBに保存
source = "bay_financials_2025.csv"
csv_path = "./.csv/bay_financials_2025_1.csv" 
# コレクション内の全データを取得
all_data = collection.get()

# CSVファイルへ出力
with open("./.csv/chroma_data.csv", "w", encoding="utf_8_sig", newline="") as f:
    writer = csv.writer(f)
    # ヘッダーの書き込み
    writer.writerow(["ID", "Document", "Metadata"])
    
    # データの書き込み
    for i in range(len(all_data["ids"])):
        writer.writerow([
            all_data["ids"][i],
            all_data["documents"][i],
            all_data["metadatas"][i]
        ])

print("CSVファイルへの出力が完了しました。")


CSVファイルへの出力が完了しました。


## 特定データの削除

In [7]:
# bay_ir_2025.pdf::image::000000 ～ 000036 を生成
ids_to_delete = [f"bay_ir_2025.pdf::image::{i:06d}" for i in range(37)]

collection.delete(ids=ids_to_delete)

print("deleted:", len(ids_to_delete))

deleted: 37
